In [1]:
import os
import csv
import pandas as pd
import numpy as np
from datetime import datetime

from scripts.sample_db import SampleDB
from skimage import measure
import tifffile

def create_bigwarp_csv_from_centroids(filename, centroids, moving="EM", fixed="LM"):
    columns = ['Point_id', 'Active', 'moving_x', 'moving_y', 'moving_z', 'fixed_x', 'fixed_y', 'fixed_z']
    df = pd.DataFrame(columns=columns)
    for i, centroid in enumerate(centroids):
        point_id = f"Pt-{i}"
        if moving == "EM":
            row = [point_id, "false"] + list(centroid) + ["Infinity", "Infinity", "Infinity"]
        else:
            row = [point_id, "false", "Infinity", "Infinity", "Infinity"] + list(centroid)
        df.loc[len(df)] = row
    df.to_csv(filename, index=False, header=None, quoting=csv.QUOTE_ALL)
    
def read_landmarks_csv(file_path, chunksize=None):
    df = pd.read_csv(file_path, header=None, 
                     names=['Point_id', 'Active', 'moving_x', 'moving_y', 'moving_z', 'fixed_x', 'fixed_y', 'fixed_z'],
                     quoting=csv.QUOTE_ALL,
                     dtype={'Point_id': str, 'Active': str, 'moving_x': str, 'moving_y': str, 'moving_z': str, 
                            'fixed_x': str, 'fixed_y': str, 'fixed_z': str})
    return df

def merge_landmarks(df1, df2):
    last_point_id = df1['Point_id'].str.extract('(\d+)').astype(int).max()
    # Update all Point_ids by adding (last_point_id + 1)
    df2['Point_id'] = 'Pt-' + (df2['Point_id'].str.extract('(\d+)').astype(int) + last_point_id + 1).astype(str)
    
    merged_landmarks = pd.concat([df1, df2])
    return merged_landmarks

def chunk_unmatched_landmarks(file_path, chunk_size=3000):
    # Read the CSV file into a DataFrame
    df = read_landmarks_csv(file_path)
    # Yield chunks of DataFrame
    for i in range(0, len(df), chunk_size):
        yield df.iloc[i:i + chunk_size]
        
def switch_fix_mov_columns(df):
    df = df[['Point_id', 'Active', 'fixed_x', 'fixed_y', 'fixed_z', 'moving_x', 'moving_y', 'moving_z']]
    df = df.rename({'fixed_x':'moving_x', 'fixed_y':'moving_y', 'fixed_z':'moving_z', 'moving_x':'fixed_x', 'moving_y':'fixed_y', 'moving_z':'fixed_z'}, errors='raise', axis=1)
    return df

    
def save_landmarks_csv(df,filename):
    df.to_csv(filename, index=False, header=False, quoting=csv.QUOTE_ALL)



In [2]:
# Load the sample database
db_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\sample_db.csv'
sample_db = SampleDB()
sample_db.load(db_path)

# Get the sample
exp = sample_db.get_sample('20220511_RM0008_126hpf_fP10_f2')

# Update EM paths
em_stack_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\CLEM_Analyses\CLEM_20220511_RM0008_126hpf_fP10_f2\fine_aligned_em_stack_20220511_RM0008_126hpf_fP10_f2_ds4_woResin_144nm_px_from15.tif'

em_mask_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\CLEM_Analyses\CLEM_20220511_RM0008_126hpf_fP10_f2\fine_aligned_em_stack_20220511_RM0008_126hpf_fP10_f2_ds4_woResin_144nm_px_from15_cp_masks_ft04_cp-3.tif'

exp.em_stack_path = em_stack_path
exp.em_mask_path = em_mask_path

# Create BigWarp CSV files
bigwarp_path = os.path.join(exp.paths.root_path, "bigwarp_alignment")
os.makedirs(bigwarp_path, exist_ok=True)

# Save the updated database
print(f"Sample database updated and saved to {db_path}")

# Define file paths of unmatched files
unmatched_landmarks_mov_EM_fix_LM_file =  f'unmatched_landmarks_mov_EM_fix_LM_{exp.sample.id}.csv'
unmatched_landmarks_mov_LM_fix_EM_file =  f'unmatched_landmarks_mov_LM_fix_EM_{exp.sample.id}.csv'

Sample database updated and saved to \\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\sample_db.csv


# Step 1: Load EM mask, calculate centroids, create and chunk unmatched landmark files, create centroids LUT.

In [4]:

# Load EM stack
em_mask = tifffile.imread(em_mask_path)
em_props = measure.regionprops(em_mask)
em_centroids = np.array([prop.centroid for prop in em_props])


new_landmarks = em_centroids[:, ::-1]  # Flip coordinates to match coordinate sequence of BigWarp
selected_indices = np.random.choice(len(em_centroids), len(em_centroids), replace=False)
selected_centroids = new_landmarks[selected_indices].astype(str)
new_landmarks_rnd = selected_centroids.tolist()



create_bigwarp_csv_from_centroids(os.path.join(bigwarp_path, unmatched_landmarks_mov_LM_fix_EM_file), new_landmarks_rnd, fixed="LM")
create_bigwarp_csv_from_centroids(os.path.join(bigwarp_path, unmatched_landmarks_mov_EM_fix_LM_file), new_landmarks_rnd, fixed="EM")

# Create and save centroids lookup table
centroid_list = [tuple(centroid) for centroid in em_centroids]
centroids_lut = pd.DataFrame({
    'id': range(1, len(centroid_list) + 1),
    'em_centroids_zyx_px': centroid_list
})
centroids_lut.to_csv(os.path.join(bigwarp_path, 'lut_centroids.csv'), index=False)

In [4]:

# Create chunks of unmatched landmarks
chunks = chunk_unmatched_landmarks(os.path.join(bigwarp_path,unmatched_landmarks_mov_EM_fix_LM_file))
# Save each chunk as a separate CSV file
for i, chunk in enumerate(chunks):
    # Create the new filename with the chunk number as prefix
    filename = os.path.join(bigwarp_path, f"chunk_{i}_{unmatched_landmarks_mov_EM_fix_LM_file}")
    save_landmarks_csv(chunk, filename)

# Step 2: Find rough landmarks with BigWarp
* Load EM stack and LM mask (with both channels if available)
* Explore and create rough_landmarks_mov_EM_fix_LM_20220511_sample_id.csv

# Step 3: Add unmatched landmarks to rough landmarks

In [3]:
# Read rough landmarks csv file
rough_landmarks_path = os.path.join(bigwarp_path, f'fine_landmarks_mov_LM_fix_EM_{exp.sample.id}.csv')
rough_landmarks = read_landmarks_csv(rough_landmarks_path)

mov_LM_fix_EM = "mov_LM_fix_EM" in rough_landmarks_path

n = 6
new_landmarks_path = os.path.join(bigwarp_path, f"chunk_{n}_{unmatched_landmarks_mov_EM_fix_LM_file}")
new_landmarks = read_landmarks_csv(new_landmarks_path)

if mov_LM_fix_EM:
    new_landmarks = switch_fix_mov_columns(new_landmarks)
    output_substring = "fine_landmarks_mov_LM_fix_EM"

else:
    output_substring = "fine_landmarks_mov_EM_fix_LM"

# Create output filename with timestamp
now = datetime.now()
formatted_datetime = now.strftime('%Y%m%d_%H%M%S')
output_file = os.path.join(bigwarp_path, f'{formatted_datetime}_{output_substring}_{exp.sample.id}.csv')

merged_landmarks = merge_landmarks(rough_landmarks[rough_landmarks["Active"] == "true"], new_landmarks)

save_landmarks_csv(merged_landmarks,output_file)

print(f"File created at {output_file}")

File created at \\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-05-11\f2\bigwarp_alignment\20241023_132221_fine_landmarks_mov_LM_fix_EM_20220511_RM0008_126hpf_fP10_f2.csv


In [4]:
# After fine alignment switch the mov LM to fix LM to get the warped EM stack in BigWarp

# Read rough landmarks csv file
final_landmarks_path = os.path.join(bigwarp_path, f'fine_landmarks_mov_LM_fix_EM_{exp.sample.id}.csv')
final_landmarks = read_landmarks_csv(final_landmarks_path)

mov_LM_fix_EM = "mov_LM_fix_EM" in final_landmarks_path
print(mov_LM_fix_EM)
final_landmarks_switched = switch_fix_mov_columns(final_landmarks)

True


In [5]:
final_landmarks.head()

,Point_id,Active,moving_x,moving_y,moving_z,fixed_x,fixed_y,fixed_z
0,Pt-25,true,363.9445824866283,132.99577150983203,17.054340413748665,1133.2430547720871,601.4250768869815,468.9082320745107
1,Pt-28,true,52.14850524427385,93.80106688473383,63.735674012180205,396.3175622435242,567.620247498691,705.9781873709779
2,Pt-236,true,347.19933092508325,222.38903158760263,98.60927481831601,1046.002568618817,815.3448554234552,283.36980772053425
3,Pt-373,true,48.802704768749194,173.31730784300294,89.03287801175833,340.3053282226596,665.14789875341,505.25201146448427
4,Pt-1804,true,427.6857016435371,90.23241740156004,60.55589273562157,1266.6205592731626,720.6956095968663,592.1766225994234


In [7]:
final_landmarks_switched.head()
final_landmarks_switched_path = os.path.join(bigwarp_path, f'fine_landmarks_mov_EM_fix_LM_{exp.sample.id}.csv')
save_landmarks_csv(final_landmarks_switched,final_landmarks_switched_path)
